In [1]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import mysql.connector as sql
import tkinter

In [3]:
root = tkinter.Tk()
root.wm_title("Regresion lineal de temperatura")
fig = Figure(figsize=(7, 7), dpi=65)
t = np.arange(0, 3, .01)

In [4]:
regr = linear_model.LinearRegression()
regr2 = linear_model.LinearRegression()

In [5]:
#conectar con la base de datos
db_connection = sql.connect(host='localhost', database='iot', user='root', password='8990')
db_cursor = db_connection.cursor()
db_cursor.execute('SELECT * FROM co2pm25')

In [6]:
#convertir de sql a dataframe
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows)

In [7]:
muestra = 'Muestra'
fecha = 'Fecha'
co2 = 'PPM CO2'
pm25 = 'ug/ml PM2.5'

In [8]:
#df.set_index(1)
df.columns = [muestra,fecha,co2,pm25]

#Convertir columnas 2 y 3 de tipo object a int64
df[co2] = pd.to_numeric(df[co2])
df[pm25] = pd.to_numeric(df[pm25])

#print(df.tail(4)) #mostrar ultimas 4 filas
print(df)
df.info()

     Muestra               Fecha   PPM CO2  ug/ml PM2.5
0          1 2021-02-19 13:21:29  843.4495     15.78826
1          2 2021-02-19 13:22:29  749.1918     86.60873
2          3 2021-02-19 13:22:59  662.9549    240.27890
3          4 2021-02-19 13:23:29  547.6113     34.69557
4          5 2021-02-19 13:24:00  512.6740    103.55980
..       ...                 ...       ...          ...
497      498 2021-02-19 17:37:20  146.0646     85.71591
498      499 2021-02-19 17:37:50  146.0646    158.94610
499      500 2021-02-19 17:38:21  132.8217     91.50940
500      501 2021-02-19 17:38:52  160.2321     48.34917
501      502 2021-02-19 17:39:23  146.0646    107.45540

[502 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Muestra      502 non-null    int64         
 1   Fecha        502 non-null    datetime64[ns]
 2  

In [9]:
x = df[muestra] #lectura cada 30s
x2 = df[fecha] #fecha
y = df[co2] #CO2
y2 = df[pm25] #PM2.5

In [10]:
#pone en formato de un arreglo, toma todos los datos y los convnierte en una matriz
X = x[:,np.newaxis]
#print(X)

In [11]:
#entrenar modelo, pasar datos de entrenamiento X,y
print(regr.fit(X,y))
print(regr2.fit(X,y2))

LinearRegression()
LinearRegression()


In [12]:
print(regr.coef_) #saber el valor de m
m = regr.coef_[0]
b = regr.intercept_
print('y={0}*x+{1}'.format(m,b))

[-0.34174518]
y=-0.3417451846845307*x+285.1642686294344


In [13]:
#hacer una prediccion de los primeros 5 valores, se toman valore de prueba para evaluar el modelo
pred1 = regr.predict(X)
pred2 = regr2.predict(X)

In [14]:
y_p = m*X+b
print("El valor de r^2: ", r2_score(y,y_p))

El valor de r^2:  0.4940493610781209


In [17]:
#añadir dos graficos en una misma ventana
spec2 = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
fig1 = fig.add_subplot(spec2[0, 0])
fig1.scatter(x,y,color = 'c')
fig1.plot(x,pred1,color = 'indigo')
fig1.set_title('Sensor de CO2')
fig1.set(xlabel='Muestras', ylabel='PPM')
fig2 = fig.add_subplot(spec2[0, 1])
fig.add_subplot(spec2[0, 1]).scatter(x,y2,color = 'g')
fig.add_subplot(spec2[0, 1]).plot(x,pred2,color = 'b')
fig2.set_title('Sensor de particulas PM2.5')
fig2.set(xlabel='Muestras', ylabel='ug/ml')
fig3 = fig.add_subplot(spec2[1, 0])
fig3.set_title('Gráfico de densidad')
sns.set_color_codes()
sns.distplot(df[co2],ax=fig3,color="y",hist=False)
sns.distplot(df[pm25],ax=fig3,color="r",hist=False)
fig3.legend(labels=['PPM CO2','ug/ml PM2.5'])
fig3.set(xlabel='Muestras', ylabel='Densidad')
fig4 = fig.add_subplot(spec2[1, 1])
fig4.set_title('Correlación entre los datos')
sns.heatmap(df.corr(),annot=True,ax=fig4)

<ipython-input-17-b752653281f6>:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  fig.add_subplot(spec2[0, 1]).scatter(x,y2,color = 'g')
<ipython-input-17-b752653281f6>:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  fig.add_subplot(spec2[0, 1]).plot(x,pred2,color = 'b')


In [18]:
canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
canvas.draw()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

toolbar = NavigationToolbar2Tk(canvas, root)
toolbar.update()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)


def on_key_press(event):
    print("you pressed {}".format(event.key))
    key_press_handler(event, canvas, toolbar)


canvas.mpl_connect("key_press_event", on_key_press)


def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate


button = tkinter.Button(master=root, text="Quit", command=_quit)
button.pack(side=tkinter.BOTTOM)




tkinter.mainloop()


TclError: can't invoke "canvas" command: application has been destroyed